In [ ]:
本节内容：
将所有样本（变量：堵塞位置、堵塞大小、用水需求）一起训练，标签为堵塞的管道。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.optim import Adam,AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from prefetch_generator import BackgroundGenerator
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib
import matplotlib.pyplot as plt 
import hiddenlayer as hl
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer,RobustScaler
from sklearn.metrics import accuracy_score
import lightgbm as lgb
#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')
#判断是否可以使用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,StratifiedKFold,cross_validate
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform,uniform,randint
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')

# ANN

In [ ]:
class DataLoaderX(DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())
#ANN
class ANN(nn.Module):
    def __init__(self,d1,hidden_size,d2):
        super(ANN,self).__init__()
        self.ann=nn.Sequential(nn.BatchNorm1d(d1,momentum=0.5),
                               nn.Linear(d1,hidden_size),
                               nn.LeakyReLU(),
                               nn.BatchNorm1d(hidden_size,momentum=0.5),
                               nn.Linear(hidden_size,d2),
                               nn.LeakyReLU()
                              )
    def forward(self,x):
        x = self.ann(x)
        return x
#训练模型
def anntrain(args):
    model=ANN(d1=args['d1'],hidden_size=args['hidden_size'],d2=args['d2']).to(device)
    criterion=nn.CrossEntropyLoss().to(device)
    optimizer=AdamW(model.parameters(),lr=args['learning_rate'],weight_decay=args['weight_decay'])
    scheduler=LambdaLR(optimizer,verbose=False,lr_lambda=lambda epoch:1/(epoch/100+1))
    train_loader=DataLoader(dataset=train_data,batch_size=len(x_train),
                                  shuffle=True,num_workers=0)
    history1=hl.History()
    canvas1=hl.Canvas()
    best_acc=0
    for epoch in tqdm(range(args['epoches'])):
        for step,(batch_x,batch_y) in enumerate(train_loader):
            #输出层与标签之间的误差    
            model.train()
            prediction=model(batch_x)
            loss=criterion(prediction,batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #记录误差曲线和acc曲线
            steps=epoch*len(train_loader)+step+1
            if steps%25==0:
                test_acc=score_model(x_test_tensor,y_test_tensor,model)
                best_acc=test_acc if test_acc>best_acc else best_acc
                history1.log(steps,train_loss=loss.item(),test_accuracy=test_acc)
        scheduler.step()
    with canvas1:
        canvas1.draw_plot(history1['train_loss'])
        canvas1.draw_plot(history1['test_accuracy'])
    print('train_loss:%.4f'%loss.item())
    print('test_acc:%.4f'%test_acc)
    print('best_acc:%.4f'%best_acc)
    return model,best_acc
#测试数据
def run_model(data,model):
    model.eval()
    torch.no_grad()
    prediction=model(data)
    return prediction.detach()
def score_model(x_tensor,label_tensor,model):
    x=run_model(x_tensor,model)
    x=torch.argmax(x,1).unsqueeze(1)
    score=(x==label_tensor).sum().item()/len(x)
    return score

## 模型寻优

In [ ]:
#读取模型数据
sample=pd.read_csv(local_path+'\\blokagedata\\Sample_116.csv',header=0)
#划分样本和标签
x=np.array(sample.iloc[:,range(0,96)])#.astype(int)
y=np.array(sample.loc[:,'code']).reshape(-1,1).astype(int)
#训练模型

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
score_record=[]
hs_record=[]
for hs in range(100,300,10):
    params={
    "d1":96,
    "hidden_size": hs,
    "d2":116,
    "learning_rate":0.01,
    "epoches":1500,
    "weight_decay":0.01
    }
    acc=[]
    for train_index, test_index in skf.split(x,y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #qt
        qt = QuantileTransformer(output_distribution = 'uniform' )
        x_train = qt.fit_transform(x_train)
        x_test = qt.transform(x_test)
        #numpy转为张量
        x_train_tensor=torch.from_numpy(x_train).float().to(device)
        y_train_tensor=torch.from_numpy(y_train).to(device)
        x_test_tensor=torch.from_numpy(x_test).float().to(device)
        y_test_tensor=torch.from_numpy(y_test).to(device)
        train_data=Data.TensorDataset(x_train_tensor,y_train_tensor.long().squeeze())
        annmodel,best_acc=anntrain(params)
        train_acc=score_model(x_train_tensor,y_train_tensor,annmodel)
        print('train_acc:',train_acc)
        acc.append(best_acc)
        print('acc:',acc)
    ave_score=np.mean(acc)
    print('ave_acc',ave_score)
    score_record.append(ave_score)
    hs_record.append(hs)
    print('score_record:',score_record)
    print('hs_record:',hs_record)

In [ ]:
#读取模型数据
sample=pd.read_csv(local_path+'\\blokagedata\\Sample_116_percent.csv',header=0)
#划分样本和标签
x=np.array(sample.iloc[:,range(0,96)])#.astype(int)
y=np.array(sample.loc[:,'code']).reshape(-1,1).astype(int)
#训练模型

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

params={
"d1":96,
"hidden_size": 110,
"d2":116,
"learning_rate":0.01,
"epoches":1500,
"weight_decay":0.01
}
acc=[]
for train_index, test_index in skf.split(x,y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #qt
    qt = QuantileTransformer(output_distribution = 'uniform' )
    x_train = qt.fit_transform(x_train)
    x_test = qt.transform(x_test)
    #numpy转为张量
    x_train_tensor=torch.from_numpy(x_train).float().to(device)
    y_train_tensor=torch.from_numpy(y_train).to(device)
    x_test_tensor=torch.from_numpy(x_test).float().to(device)
    y_test_tensor=torch.from_numpy(y_test).to(device)
    train_data=Data.TensorDataset(x_train_tensor,y_train_tensor.long().squeeze())
    annmodel,best_acc=anntrain(params)
    train_acc=score_model(x_train_tensor,y_train_tensor,annmodel)
    print('train_acc:',train_acc)
    acc.append(best_acc)
    print('acc:',acc)
    break

## 处理结果

In [ ]:
torch.save(annmodel.state_dict(),local_path+'\\blokagedata\\annmodel_hid110.pkl')
# model = TheModelClass(...)
# model.load_state_dict(torch.load('\parameter.pkl'))

In [ ]:
# model = TheModelClass(...)
# model.load_state_dict(torch.load('\parameter.pkl'))
pre=run_model(x_test_tensor,annmodel)
pre=torch.argmax(pre,1).unsqueeze(1).cpu().numpy()
bool1=np.ones([len(pre),1])
bool1[np.where(pre.ravel()!=y_test.ravel())]=0
pred=pd.DataFrame(pre)
S=sample.loc[test_index,:]
para=S.iloc[:,range(96,99)].reset_index()
bool2=pd.DataFrame(bool1.astype(int))
a=pd.concat([pred,para,bool2],axis=1,ignore_index=True)
a.columns=['pred','index','true','dn','time','bool']
a=a[['index','pred','true','dn','time','bool']]
a.to_csv(local_path+'\\blokagedata\\ann_hid110_test_result.csv',encoding='ascii',index=True)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
cm=pd.DataFrame(confusion_matrix(y_test.ravel(),pre))
cm.to_csv(local_path+'\\blokagedata\\ann_hid110_cm.csv',encoding='ascii',index=True)

# SVM

In [ ]:
#读取模型数据
sample=pd.read_csv(local_path+'\\blokagedata\\Sample_116.csv',header=0)
#划分样本和标签
x=np.array(sample.iloc[:,range(0,96)])#.astype(int)
y=np.array(sample.loc[:,'code']).reshape(-1,1).astype(int)

## 模型寻优

In [ ]:
pipe = Pipeline([('scaler', QuantileTransformer(output_distribution = 'uniform')), ('svc', SVC(decision_function_shape='ovr',kernel='rbf'))])
param_dist = {'svc__C': [1,10,100,1000,10000,100000,500000,800000,1000000,1200000,1500000],
 'svc__gamma': [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.01]}
splitter=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
clf = GridSearchCV(estimator=pipe,param_grid=param_dist,cv=splitter,n_jobs=-1,scoring='accuracy',verbose=3)
search = clf.fit(x, y.ravel())

In [ ]:
print('网格搜索-度量记录：',search.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',search.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',search.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',search.best_estimator_)  # 获取最佳度量时的分类器模型

## 处理结果

In [ ]:
xx=search.cv_results_['param_svc__C'].data.astype(float)
yy=search.cv_results_['param_svc__gamma'].data.astype(float)
zz=search.cv_results_['mean_test_score'].astype(float)
xx=np.expand_dims(xx,axis=1)
yy=np.expand_dims(yy,axis=1)
zz=np.expand_dims(zz,axis=1)
print(xx.shape)
print(yy.shape)
print(zz.shape)
zxc=pd.DataFrame(np.hstack((xx,yy,zz)))
zxc.columns=['C','gamma','acc']
zxc=pd.pivot_table(zxc,index=['C'],columns=['gamma'],values=['acc'])
zxc.to_csv(local_path+'\\blokagedata\\svm_finetune_grid_b.csv',encoding='ascii',index=True)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123,stratify=y)
best_clf=search.best_estimator_
y_pre=best_clf.predict(x_test)
cm=pd.DataFrame(confusion_matrix(y_test.ravel(),y_pre))
cm.to_csv(local_path+'\\blokagedata\\svm_cm.csv',encoding='ascii',index=True)

# RF

## 模型寻优

In [ ]:
pipe = Pipeline([('scaler', QuantileTransformer(output_distribution = 'uniform')), ('rfc', RandomForestClassifier(criterion='gini',))])
param_dist = {'rfc__n_estimators': randint(10, 500),
 'rfc__min_samples_split': randint(2, 1000),
 'rfc__max_depth': randint(2, 100)}
splitter=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
clf = RandomizedSearchCV(estimator=pipe,param_distributions=param_dist,n_iter=10,cv=splitter,random_state=11,n_jobs=-1,scoring='accuracy',verbose=3)
search = clf.fit(x, y.ravel())

In [ ]:
print('网格搜索-度量记录：',search.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',search.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',search.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',search.best_estimator_)  # 获取最佳度量时的分类器模型

## 处理结果

In [ ]:
q1=search.cv_results_['param_rfc__n_estimators'].data.astype(int)
q2=search.cv_results_['param_rfc__min_samples_split'].data.astype(int)
q3=search.cv_results_['param_rfc__max_depth'].data.astype(int)
zz=search.cv_results_['mean_test_score'].astype(float)
q1=np.expand_dims(q1,axis=1)
q2=np.expand_dims(q2,axis=1)
q3=np.expand_dims(q3,axis=1)
zz=np.expand_dims(zz,axis=1)
print(q1.shape)
print(q2.shape)
print(q3.shape)
print(zz.shape)
pd.DataFrame(np.hstack((q1,q2,q3,zz))).to_csv(local_path+'\\blokagedata\\rfc_finetune.csv',encoding='ascii',index=False)

In [ ]:
ave=[]
m=[]
for n in tqdm(range(50,150,10)):
    rfc=RandomForestClassifier(n_estimators=n,criterion='gini',max_depth=None,min_samples_split=2,n_jobs=-1)
    scores = cross_validate(rfc, x, y.ravel(), cv=splitter,scoring=('accuracy'),return_train_score=True,n_jobs=-1)
    m.append(n)
    ave.append(np.mean(scores['test_score']))

In [ ]:
rfc=RandomForestClassifier(n_estimators=150,criterion='gini',max_depth=None,min_samples_split=2,n_jobs=-1)
rfc.fit(x_train,y_train.ravel())
score=rfc.score(x_test,y_test.ravel())
print('rfc:',score)

# KNN

## 模型寻优

In [ ]:
pipe = Pipeline([('scaler', QuantileTransformer(output_distribution = 'uniform')), ('knn', KNeighborsClassifier())])
param_dist = {'knn__n_neighbors': [1,2,3,4,5,6,7,8,9]}
splitter=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
clf = RandomizedSearchCV(estimator=pipe,param_distributions=param_dist,n_iter=9,cv=splitter,random_state=1,n_jobs=-1,scoring='accuracy',verbose=3)
search = clf.fit(x, y.ravel())

In [ ]:
print('网格搜索-度量记录：',search.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',search.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',search.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',search.best_estimator_)  # 获取最佳度量时的分类器模型

## 处理结果

In [ ]:
xx=search.cv_results_['param_knn__n_neighbors'].data.astype(float)
zz=search.cv_results_['mean_test_score'].astype(float)
xx=np.expand_dims(xx,axis=1)
zz=np.expand_dims(zz,axis=1)
print(xx.shape)
print(zz.shape)
zxc=pd.DataFrame(np.hstack((xx,zz)))
zxc.columns=['k','acc']
zxc.to_csv(local_path+'\\blokagedata\\knn_finetune.csv',encoding='ascii',index=True)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123,stratify=y)
knn=KNeighborsClassifier(n_neighbors=1, weights='distance',metric='minkowski', p=2)
knn.fit(x_train,y_train.ravel())
score=knn.score(x_test,y_test.ravel())
print('knn:',score)